In [1]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import pandas as pd
import rich as rc
import os

In [2]:
data = pd.read_csv("../data/DicatJuanRana.csv", sep=";")

In [3]:
data.head()

,year,text
0,0,"Aunque su verdadero nombre era Cosme Pérez, fu..."
1,1617,"Según Cotarelo , a quien sigue Sáez Raposo , C..."
2,1621,Consta la lista de la compañía de Juan Bautist...
3,1624,Cosme Pérez aparece en la nómina de la compañí...
4,1634,"Según Cotarelo, Cosme Pérez pertenecía a la co..."


In [4]:
load_dotenv()

llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0,
    openai_api_key=os.getenv("OPENAI_API_KEY"),
)

In [14]:
prompt = PromptTemplate(
    input_variables=["text"],
    template="""
    Eres un asistente de IA experto en el análisis de texto. Tu tarea es separar un texto en frases individuales.
    Cada frase debe ser una unidad completa de significado pero no necesariamente debe terminar en un punto.
    Se trata de un texto en español antiguo, por lo que no se usaban las reglas modernas de puntuación.
    Por favor, asegúrate de que cada frase sea coherente y tenga sentido por sí misma.
    Asegurate de no inventar o perder información, juntando las frases deberían formar nuevamente el texto original.
    
    NO DEBES INVENTAR O EXCLUIR INFORMACIÓN, SOLO SEPARAR EN FRASES.
     
    Devuelve cada frase en una línea separada:
    {text}""",
)

In [15]:
from tqdm.auto import tqdm

all_sentences = []
for i, row in tqdm(data.iterrows(), total=len(data)):
    text = row["text"]
    response = llm.invoke(prompt.format(text=text))
    sentences = response.content.split("\n")
    all_sentences.append(sentences)

  0%|          | 0/38 [00:00<?, ?it/s]

In [16]:
len(all_sentences)

38

In [17]:
data["sentences"] = all_sentences

In [18]:
data.to_csv(
    "../data/DicatJuanRana_w_sentences.csv",
    sep=";",
    index=False,
)

In [24]:
data[data["year"].astype(int) == 1643]["sentences"].to_list()

[['El 27 de marzo —según Davis y Varey, y no el 28 de febrero, como lee Pérez Pastor— Catalina de Nicolás y la Rosa, en virtud del poder del 17 de febrero otorgado por su marido, Pedro de la Rosa, contrató en Madrid a Cosme Pérez, desde la fecha del contrato hasta el día del Corpus, 4 de junio, y, si fuese necesario para el autor, para los días de la Octava del Corpus, representando "su parte de toda la graciosidad" en comedias, autos, bailes y entremeses, como lo había hecho "en otros años antezedentes a este asiento", por lo que cobraría 12 rs. de ración, 20 rs. por representación, 50 dcs. por el Corpus, las caballerías "que siempre se le an dado" para el transporte y 1.000 rs. de préstamo.  ',
  'En nombre de su marido, Catalina se comprometía asimismo a pagar 456 rs. del viaje de Murcia, de hato y personas, y las raciones de Cuaresma ; según Cotarelo, la compañía de Pedro de la Rosa representó en Madrid los autos del Corpus [4 de junio] y en la lista de actores presentada a las aut

In [26]:
import pandas as pd
import ast
    
data = pd.read_csv("../data/DicatJuanRana_w_clarified_sentences.csv", sep=";", encoding="utf-8", converters={"sentences": ast.literal_eval, "clarified_sentences": ast.literal_eval})

In [25]:
sum([len(sentences) for sentences in data["sentences"]])

358

In [27]:
sum([len(sentences) for sentences in data["sentences"]])

311